# Fake News Detection Based on Hybrid Model (CNN & LSTM) and Simple RNN 
Simple Implementation based on the PDF documant in the repository.
you can find the material on this [link](https://www.sciencedirect.com/science/article/pii/S2667096820300070https://www.sciencedirect.com/science/article/pii/S2667096820300070)

## Read Data

## Preprocessing